# Backtesting the HMM Model

Signals are identifed by the hmm model and imported into the system. 

    When signal = 1, we enter the mean reversion position.    
    When signal = 0, we enter the momentum position. 


Mean Reversion Strategy:

     1.If return in the past 21 days > 0.02, we sell     
     2.If return in the past 21 days < -0.02, we buy     
     3.We close the position every 10 days 


Momentum Strategy: 

    1.If return in the past 6 months > 0.05, we sell
    2.If return in the past 6 months < -0.05, we buy
    3.We close the position every month 
    
    
    


In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

import pandas as pd
import datetime

In [23]:

class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

        
class DummyInd(bt.Indicator):
    lines = ('dummyline',)

    params = (('value', 5),)

    def __init__(self):
        self.lines.dummyline = bt.Max(0.0, self.params.value)

        
# Create a Stratey
class BackTradingStrategy(bt.Strategy):
    params = (
        ('period', 128),
        ("max_days_in_trade", 10)
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
        
    def read_signals(self):
        signal_data = pd.read_csv("hmm_regime_signals.csv")
        signal_data["datetime"] = signal_data["datetime"].apply(lambda x: datetime.datetime.strptime(x,"%m/%d/%Y"))

        signal_data = signal_data.set_index('datetime')
        
        """
        start_date = '2013-02-11'
        end_date = '2018-02-09'
        signals = signal_data['state'].loc[start_date:end_date]               
        dt = self.datadate(0).strftime('%Y-%m-%d')
        print(signal_data.iloc[0])
        print(signal_data['state'].loc[dt])
        """
        
        return signal_data
        

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        # How to access individual etf
        self.xlb_close = self.datas[symbols.index("XLB")].close
        self.xle_close = self.datas[symbols.index("XLE")].close
        self.xlf_close = self.datas[symbols.index("XLF")].close
        self.xli_close = self.datas[symbols.index("XLI")].close
        self.xlk_close = self.datas[symbols.index("XLK")].close
        self.xlp_close = self.datas[symbols.index("XLP")].close
        self.xlu_close = self.datas[symbols.index("XLU")].close
        self.xlv_close = self.datas[symbols.index("XLV")].close
        self.xly_close = self.datas[symbols.index("XLY")].close
        
        self.datadate = self.datas[0].datetime.date
        self.days_in_trade = 0
        self.max_days_in_trade_mr = 10
        self.max_days_in_trade_mo = 21
        self.num_mr = 21
        self.num_mo = 21*6
        self.regime_state = 0
        self.mrr = 0.02
        self.mor = 0.05
        self.signal_data = self.read_signals()
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)        
        self.symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
        
        
        
    def stock_return(self, n):
        """calculate return in the past n days """        
        r_xlb = (self.xlb_close[0] - self.xlb_close[-n])/self.xlb_close[-n]
        r_xle = (self.xle_close[0] - self.xle_close[-n])/self.xle_close[-n]
        r_xlf = (self.xlf_close[0] - self.xlf_close[-n])/self.xlf_close[-n]
        r_xli = (self.xli_close[0] - self.xli_close[-n])/self.xli_close[-n]
        r_xlk = (self.xlk_close[0] - self.xlk_close[-n])/self.xlk_close[-n]
        r_xlp = (self.xlp_close[0] - self.xlp_close[-n])/self.xlp_close[-n]
        r_xlu = (self.xlu_close[0] - self.xlu_close[-n])/self.xlu_close[-n]
        r_xlv = (self.xlv_close[0] - self.xlv_close[-n])/self.xlv_close[-n]
        r_xly = (self.xly_close[0] - self.xly_close[-n])/self.xly_close[-n]
        r_list = [r_xlb, r_xle, r_xlf, r_xli, r_xlk, r_xlp, r_xlu, r_xlv, r_xly]        
        
        df = pd.DataFrame(r_list, index = self.symbols, columns = ['return'])
        
        return df
    
    
    
    def sort_return(self, n):
        """sort return in the past n days in ascending order 
           return the bottom and top 3 return stocks """
        
        df = self.stock_return(n)
        df = df.sort_values(by=['return'])
        stock_list = [df.index[0:3].tolist(),df.index[6:].tolist()]
        
        return stock_list
    
    
    
    def check_position(self):
        flag = False
        name = 'none'
        for symbol in self.symbols:
            s_size = self.getposition(data=self.datas[symbols.index(symbol)]).size
            #print(s_size)
            if  s_size != 0:
                flag = True
                name = symbol
                break
        return flag, name
            
                
    

    def next(self):
        # Simply log the closing price of the series from the reference
        
        self.log('Close, %.2f' % self.xlb_close[0])        
        
        dt = self.datadate(0).strftime('%Y-%m-%d')
        
        signal = self.signal_data['state'].loc[dt]
        
        position, name = self.check_position()
        #print('name = ', name)
        
        # Check if we are in the market
        if not position:
            # if we do not have a position
            
            if signal == 1:
                # mean reversion
                # we BUY if return in the past num_mr days is at the bottom
                
                stock_list = self.sort_return(self.num_mr)
                
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.xlb_close[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(data = self.datas[symbols.index(stock_list[0][0])])

                self.days_in_trade = 1

                self.regime_state = 1
                    
                    
         
            
            else:
                # momentum
                # we SELL if return in the past num_mo days is less than -mor
                
                stock_list = self.sort_return(self.num_mo)

                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.xlb_close[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell(data= self.datas[symbols.index(stock_list[0][0])])

                self.days_in_trade = 1

                self.regime_state = 0
                    
            
                
            

        else:
            # if we have a position
            
            self.days_in_trade += 1
            
            # mean reversion
            if (self.regime_state == 1):
            
                # close the position if regime shift or after holding it for max_mr days.
                if signal == 0 or self.days_in_trade == self.max_days_in_trade_mr:
                    
                    if self.getposition(data=self.datas[symbols.index(name)]).size > 0:
                        # SELL to close position
                        self.log('Close poition---SELL EMPTY, %.2f' % self.xlb_close[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.sell(data = self.datas[symbols.index(name)])
                        
                        self.days_in_trade = 0                        
                        self.regime_state = 0
                
                    else:
                        # BUY to close position 
                        self.log('Close poition---BUY EMPTY, %.2f' % self.xlb_close[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy(data = self.datas[symbols.index(name)])
                        
                        self.days_in_trade = 0                        
                        self.regime_state = 0
            
            
            # momentum
            if (self.regime_state == 0):
                
                # close the position if regime shift or after holding it for max_mo days.
                if signal == 1 or self.days_in_trade == self.max_days_in_trade_mo:
                    
                    if self.getposition(data=self.datas[symbols.index(name)]).size > 0:
                        # SELL to close position
                        self.log('Close poition---SELL EMPTY, %.2f' % self.xlb_close[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.sell(data= self.datas[symbols.index(name)])
                        
                        self.days_in_trade = 0                        
                        self.regime_state = 1
                
                    else:
                        # BUY to close position 
                        self.log('Close poition---BUY EMPTY, %.2f' % self.xlb_close[0])
                        #print('stock = ', stock_list[0][0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy(data= self.datas[symbols.index(name)])
                        
                        self.days_in_trade = 0                        
                        self.regime_state = 1
                 
       


In [24]:


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()   

    symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
    for symbol in symbols:
        datapath = os.path.join('../../../datas/%s.csv' % symbol)

        # Create a Data Feed
        data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=datetime.datetime(2013, 2, 11),
            # Do not pass values before this date
            todate=datetime.datetime(2018, 2, 9),
            # Do not pass values after this date
            reverse=False)

        # Add the Data Feed to Cerebro
        cerebro.adddata(data, name=symbol)


    # Add a strategy
    cerebro.addstrategy(BackTradingStrategy) #, symbols= symbols)
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    

    # Write output
    cerebro.addwriter(bt.WriterFile, out='backtesting.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2013-08-13, Close, 37.88
2013-08-13, SELL CREATE, 37.88
2013-08-14, Close, 37.80
2013-08-15, Close, 37.53
2013-08-16, Close, 37.23
2013-08-19, Close, 36.94
2013-08-20, Close, 37.15
2013-08-21, Close, 36.87
2013-08-22, Close, 37.31
2013-08-23, Close, 37.61
2013-08-26, Close, 37.61
2013-08-27, Close, 36.95
2013-08-28, Close, 36.88
2013-08-29, Close, 37.00
2013-08-30, Close, 36.93
2013-09-03, Close, 37.15
2013-09-04, Close, 37.50
2013-09-05, Close, 37.56
2013-09-06, Close, 37.52
2013-09-09, Close, 38.08
2013-09-10, Close, 38.42
2013-09-11, Close, 38.66
2013-09-11, Close poition---BUY EMPTY, 38.66
2013-09-12, Close, 38.26
2013-09-12, SELL CREATE, 38.26
2013-09-13, Close, 38.51
2013-09-16, Close, 38.94
2013-09-17, Close, 38.83
2013-09-18, Close, 39.73
2013-09-19, Close, 39.59
2013-09-20, Close, 39.13
2013-09-23, Close, 38.84
2013-09-24, Close, 38.82
2013-09-25, Close, 38.92
2013-09-26, Close, 39.18
2013-09-27, Close, 38.71
2013-09-30, Close, 38.57
2013-10

2014-12-04, Close, 46.63
2014-12-05, Close, 46.60
2014-12-08, Close, 45.86
2014-12-09, Close, 45.98
2014-12-10, Close, 45.03
2014-12-11, Close, 45.07
2014-12-12, Close, 43.82
2014-12-15, Close, 43.58
2014-12-16, Close, 43.24
2014-12-17, Close, 44.42
2014-12-18, Close, 45.44
2014-12-19, Close, 46.00
2014-12-22, Close, 46.02
2014-12-23, Close, 46.38
2014-12-24, Close, 46.22
2014-12-26, Close, 46.38
2014-12-29, Close, 46.38
2014-12-30, Close, 46.22
2014-12-30, Close poition---BUY EMPTY, 46.22
2014-12-31, Close, 45.77
2014-12-31, SELL CREATE, 45.77
2015-01-02, Close, 45.83
2015-01-05, Close, 44.67
2015-01-06, Close, 44.27
2015-01-07, Close, 44.77
2015-01-08, Close, 45.83
2015-01-09, Close, 45.60
2015-01-12, Close, 45.50
2015-01-13, Close, 44.99
2015-01-14, Close, 44.46
2015-01-15, Close, 44.37
2015-01-16, Close, 45.08
2015-01-20, Close, 45.16
2015-01-21, Close, 45.60
2015-01-22, Close, 46.17
2015-01-23, Close, 45.46
2015-01-26, Close, 45.66
2015-01-27, Close, 45.34
2015-01-28, Close, 44.53

2016-01-26, Close, 36.50
2016-01-27, Close, 36.21
2016-01-28, Close, 36.27
2016-01-29, Close, 37.31
2016-02-01, Close, 37.47
2016-02-02, Close, 37.22
2016-02-02, Close poition---SELL EMPTY, 37.22
2016-02-02, Close poition---SELL EMPTY, 37.22
2016-02-03, Close, 38.47
2016-02-04, Close, 39.54
2016-02-05, Close, 39.09
2016-02-08, Close, 38.06
2016-02-09, Close, 38.53
2016-02-10, Close, 38.16
2016-02-11, Close, 37.33
2016-02-12, Close, 38.44
2016-02-16, Close, 39.03
2016-02-16, Close poition---BUY EMPTY, 39.03
2016-02-17, Close, 39.80
2016-02-17, SELL CREATE, 39.80
2016-02-18, Close, 39.68
2016-02-19, Close, 39.22
2016-02-22, Close, 39.97
2016-02-23, Close, 38.99
2016-02-24, Close, 39.39
2016-02-25, Close, 39.90
2016-02-26, Close, 40.48
2016-02-26, Close poition---BUY EMPTY, 40.48
2016-02-29, Close, 40.23
2016-02-29, BUY CREATE, 40.23
2016-03-01, Close, 41.26
2016-03-02, Close, 41.10
2016-03-02, Close poition---SELL EMPTY, 41.10
2016-03-03, Close, 41.35
2016-03-03, SELL CREATE, 41.35
2016-

2017-04-05, Close, 51.58
2017-04-06, Close, 51.86
2017-04-07, Close, 51.81
2017-04-10, Close, 51.82
2017-04-11, Close, 51.68
2017-04-12, Close, 51.04
2017-04-13, Close, 50.55
2017-04-17, Close, 50.91
2017-04-18, Close, 50.85
2017-04-19, Close, 50.92
2017-04-20, Close, 51.49
2017-04-20, Close poition---BUY EMPTY, 51.49
2017-04-21, Close, 51.43
2017-04-21, SELL CREATE, 51.43
2017-04-24, Close, 52.02
2017-04-25, Close, 52.85
2017-04-26, Close, 52.84
2017-04-27, Close, 52.73
2017-04-28, Close, 52.28
2017-05-01, Close, 52.31
2017-05-02, Close, 52.40
2017-05-03, Close, 51.89
2017-05-04, Close, 51.95
2017-05-05, Close, 52.69
2017-05-08, Close, 52.26
2017-05-09, Close, 51.87
2017-05-10, Close, 52.04
2017-05-11, Close, 51.96
2017-05-12, Close, 51.83
2017-05-15, Close, 52.28
2017-05-16, Close, 52.29
2017-05-17, Close, 51.19
2017-05-18, Close, 51.22
2017-05-19, Close, 51.66
2017-05-19, Close poition---BUY EMPTY, 51.66
2017-05-22, Close, 51.72
2017-05-22, SELL CREATE, 51.72
2017-05-23, Close, 51.7

In [ ]:
# calculate pnl using excel data
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('backtesting.csv').fillna(value = 0)
data = data[:1260]

df = data[['datetime', 'pnlplus', 'pnlminus']].copy()
n = len(df)

df['cumulative_pnl'] = df['pnlplus']

cpnl = 0.
count_plus = 0
count_minus = 0

for i in range(n):
    plus = df['pnlplus'].iloc[i]
    minus = df['pnlminus'].iloc[i]
    cpnl = cpnl + plus + minus
    df['cumulative_pnl'].iloc[i] = cpnl
    
    if (plus > 0.):
        count_plus +=1
    if (minus < 0.):
        count_minus +=1

    

In [ ]:
plus_percent = 1.0*count_plus/(count_plus*1.0+count_minus*1.0) 
print('percentage of trading that make positive profits = ', plus_percent)

In [ ]:
# convert string to datetime
df["datetime"] = df["datetime"].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f"))

# plot PNL figures 
plt.figure(figsize=(16, 4))
plt.plot(df['datetime'], df['cumulative_pnl'] ,'.') 
plt.xlabel("Time")
plt.ylabel("cumulative pnl")
plt.title("Daily PNL")
plt.show()